In [6]:
import pandas as pd
import json
import re
import numpy as np

In [7]:
INPUT_PATH = "../../data/detail_output.jsonl"
OUTPUT_PATH = "../data/data_origin/data.csv"

In [8]:
def _to_float_first_number(s: str):
    """Extract first number, supports comma decimal."""
    if s is None or pd.isna(s):
        return np.nan
    s = str(s).lower().strip()
    # normalize decimal comma -> dot
    s = s.replace(",", ".")
    m = re.search(r"(\d+(\.\d+)?)", s)
    return float(m.group(1)) if m else np.nan

def _to_float_from_range_or_first(s: str):
    """
    If value looks like a range '6.5 - 6.9' or '6.5 ~ 6.9', return average.
    Else return first number.
    """
    if s is None or pd.isna(s):
        return np.nan
    t = str(s).lower().strip().replace(",", ".")
    nums = re.findall(r"\d+(?:\.\d+)?", t)
    if not nums:
        return np.nan
    if ("-" in t or "~" in t) and len(nums) >= 2:
        a, b = float(nums[0]), float(nums[1])
        # average range
        return (a + b) / 2.0
    return float(nums[0])

def clean_price_to_billion(x):
    """
    Convert price string -> billion VND.
    Handles: '5,59 tỷ', '950 triệu', '6.5 - 6.9 tỷ', 'thỏa thuận' -> NaN.
    """
    if x is None or pd.isna(x):
        return np.nan
    s = str(x).lower().strip()

    # common non-numeric cases
    if any(k in s for k in ["thỏa thuận", "thoả thuận", "liên hệ", "đang cập nhật", "update"]):
        return np.nan

    val = _to_float_from_range_or_first(s)
    if pd.isna(val):
        return np.nan

    # normalize units
    if "triệu" in s:
        return val / 1000.0   # million -> billion
    if "tỷ" in s or "ty" in s:
        return val

    # If missing unit, treat as invalid (safer)
    return np.nan

def clean_area_m2(x):
    """Convert '23,5 m²' or '81 m2' -> 23.5 / 81.0"""
    if x is None or pd.isna(x):
        return np.nan
    s = str(x).lower().strip()
    s = s.replace("m²", " ").replace("m2", " ").replace("㎡", " ")
    # handle ranges like "67-72 m2" or "67 ~ 72 m2"
    return _to_float_from_range_or_first(s)

def clean_ppm2_to_million(x):
    """Convert '~98,76 triệu/m²' or '90 triệu/m2' -> 98.76 / 90.0"""
    if x is None or pd.isna(x):
        return np.nan
    s = str(x).lower().replace("~", "").strip()
    # If it's not in million, you can extend later (nghìn/triệu/tỷ...)
    return _to_float_from_range_or_first(s)

def extract_int_nullable(x):
    """Extract first integer from strings like '3 PN', '6 phòng'. Return pandas Int64 (nullable)."""
    if x is None or pd.isna(x):
        return pd.NA
    m = re.search(r"\d+", str(x))
    return int(m.group()) if m else pd.NA

# -------------------------
# Load jsonl -> df
# -------------------------
df = pd.read_json(INPUT_PATH, lines=True)

# Drop useless col
df = df.drop(columns=["coordinates_text"], errors="ignore")

# Parse dates (dd/mm/YYYY)
df["posted_date"] = pd.to_datetime(df["posted_date"], format="%d/%m/%Y", errors="coerce")
df["expired_date"] = pd.to_datetime(df["expired_date"], format="%d/%m/%Y", errors="coerce")
df["scrape_time"] = pd.to_datetime(df["scrape_time"], errors="coerce")

# -------------------------
# Numeric clean columns (units moved to column names)
# -------------------------
df["price_billion"] = df["price"].apply(clean_price_to_billion)
df["area_m2"] = df["area"].apply(clean_area_m2)
df["ppm2_million_site"] = df["price_per_m2"].apply(clean_ppm2_to_million)

# calc from cleaned price + area (often more reliable)
df["ppm2_million_calc"] = (df["price_billion"] * 1000.0) / df["area_m2"]

# rooms: only numbers
df["bedrooms_num"] = df["bedrooms"].apply(extract_int_nullable).astype("Int64")
df["bathrooms_num"] = df["bathrooms"].apply(extract_int_nullable).astype("Int64")

# -------------------------
# Optional: filter rows missing core numbers
# -------------------------
df_clean = df.dropna(subset=["district", "price_billion", "area_m2"]).copy()

# Optional: remove extreme outliers
df_clean = df_clean[
    df_clean["area_m2"].between(10, 2000) &
    df_clean["price_billion"].between(0.2, 300) &
    df_clean["ppm2_million_calc"].between(5, 5000)
].copy()

df_clean.to_csv(OUTPUT_PATH, index=False, encoding="utf-8-sig")

In [9]:
df_clean.info()

<class 'pandas.DataFrame'>
Index: 3067 entries, 0 to 3314
Data columns (total 34 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   url                      3067 non-null   str           
 1   listing_id               3067 non-null   int64         
 2   scrape_time              3067 non-null   datetime64[us]
 3   title                    3067 non-null   str           
 4   phone_number             3067 non-null   str           
 5   description              3067 non-null   str           
 6   transaction_type         3067 non-null   str           
 7   city                     3067 non-null   str           
 8   district                 3067 non-null   str           
 9   property_type            3067 non-null   str           
 10  project_name_breadcrumb  3067 non-null   str           
 11  price                    3067 non-null   str           
 12  price_per_m2             3067 non-null   str      